<a href="https://colab.research.google.com/github/gracjans/GSN/blob/main/Gracjan_Strzelec_239580_GSN_1_zima_2021_22_Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSN 1 - Winter 2021/22 

In [291]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [292]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import trange
from sklearn.metrics import accuracy_score
from tensorflow import keras

import matplotlib.pyplot as plt
import cv2

In [293]:
"""
myNetwork class parameters:
      neurons_count: int (number of neurons in layer)     
      inputs_count: int (number of inputs to each neuron of layer)
      weights: OPTIONAL - tuple of arrays (arrays of weights for each of layer)
      biases: OPTIONAL - tuple of arrays (arrays of biases for each of layer)
"""

class myLayer:
    def __init__(self, inputs_count, neuron_count, learning_rate=0.01, weights=None, biases=None):
        self.learning_rate = learning_rate

        # check if weights or biases are passed as an network argument, otherwise initialize it by default
        if weights is not None:
            self.weights = np.array(weights)  
        else:  
            self.weights = np.random.normal(scale=0.01, size=(neuron_count, inputs_count))

        if biases is not None:
            self.biases = np.array(biases)
        else:  
            self.biases = np.zeros(neuron_count)

    def forward(self, inputs):
        return np.dot(inputs, self.weights.T) + self.biases

    def backward(self, input, grad):
        grad_input = np.dot(grad, self.weights)
        grad_weights = np.dot(input.T, grad)
        grad_biases = grad.mean(axis=0) * input.shape[0]

        self.weights = self.weights - self.learning_rate * grad_weights.T
        self.biases = self.biases - self.learning_rate * grad_biases.T

        return grad_input

In [294]:
# ACTIVATION FUNCTIONS

class sigmoid():
    def forward(self, x):
      return 1 / (1 + np.exp(-X))
    def backward(self, x, grad):
      sig_grad = x * (1 - x)
      return grad * sig_grad

class relu():
    def forward(self, x):
      return x * (x > 0)
    def backward(self, x, grad):
      re_grad = 1 * (x > 0)
      return grad * re_grad

class tanh():
    def forward(self, x):
      return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    def backward(self, x, grad):
      tan_grad = 1 - (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))**2
      return grad * tan_grad

In [295]:
def softmax_crossentropy_with_logits(logits, reference_answers):
    logits_for_answers = logits[np.arange(len(logits)),reference_answers]
    xentropy = -logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
    return xentropy

def grad_softmax_crossentropy_with_logits(logits, reference_answers):
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)),reference_answers] = 1
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1, keepdims=True)
    return (- ones_for_answers + softmax) / logits.shape[0]

In [296]:
class myNetwork:
    def __init__(self):
        self.layers_instances = []

    def add_layer(self, layer):
        self.layers_instances.append(layer)

    def forward(self, inputs):
        self.inputs = [inputs]

        current_state = np.array(inputs)
        for layer in self.layers_instances:
          current_state = layer.forward(current_state)
          self.inputs.append(current_state)
        return current_state

    def backward(self, grad):
        current_state = grad
        for layer_index in range(len(self.layers_instances))[::-1]:
          layer = self.layers_instances[layer_index]
          current_state = layer.backward(self.inputs[layer_index], current_state)

In [297]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)

    if shuffle:
        indices = np.random.permutation(len(inputs)) 

    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize, desc='minibatch iteration'):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)

        yield inputs[excerpt], targets[excerpt]

### Initialize and train neural network with MNIST dataset

In [300]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_test = np.reshape(X_train / 255., (60000, 28*28)), np.reshape(X_test /255. , (10000, 28 * 28))

print(X_train.shape)
print(y_train.shape)

(60000, 784)
(60000,)


In [306]:
### full network training ###
net = myNetwork()
net.add_layer(myLayer(784, 128))
net.add_layer(tanh())
net.add_layer(myLayer(128, 64))
net.add_layer(tanh())
net.add_layer(myLayer(64, 10))

for epoch in range(10):
    epoch_loss = []
    for x_batch, y_batch in iterate_minibatches(X_train, y_train, batchsize=32, shuffle=True):

        ### forward ###
        logits = net.forward(x_batch)

        ### loss calculation ###
        loss = softmax_crossentropy_with_logits(logits, y_batch)
        epoch_loss.append(np.mean(loss))

        ### backward ###
        loss_grad = grad_softmax_crossentropy_with_logits(logits, y_batch)
        net.backward(loss_grad)

    preds = []
    for x_batch, _ in iterate_minibatches(X_test, y_test, batchsize=1, shuffle=False):
        preds.append(np.argmax(net.forward(x_batch)))

    print(f'\nEpoch: {epoch + 1} \t Loss: {np.mean(epoch_loss)} \t Test accuracy: {accuracy_score(y_test, preds)} \n')

minibatch iteration: 100%|██████████| 10000/10000 [00:01<00:00, 7925.22it/s]



Epoch: 1 	 Loss: 2.282199967214619 	 Test accuracy: 0.23 



minibatch iteration: 100%|██████████| 10000/10000 [00:01<00:00, 7733.64it/s]



Epoch: 2 	 Loss: 1.3644134803034604 	 Test accuracy: 0.7602 



minibatch iteration: 100%|██████████| 10000/10000 [00:01<00:00, 7882.34it/s]


...


Epoch: 8 	 Loss: 0.3038924750929168 	 Test accuracy: 0.9149 



minibatch iteration: 100%|██████████| 10000/10000 [00:01<00:00, 7675.76it/s]



Epoch: 9 	 Loss: 0.2882281897048434 	 Test accuracy: 0.9175 



minibatch iteration: 100%|██████████| 10000/10000 [00:01<00:00, 7899.86it/s]


Epoch: 10 	 Loss: 0.27610334161270617 	 Test accuracy: 0.9199 



### Initialize and train neural network with previously created dataset (x, o, + images)

In [309]:
X = np.load('drive/MyDrive/lab2-dataset/X.npy')
y = np.load('drive/MyDrive/lab2-dataset/y.npy')
bb = np.load('drive/MyDrive/lab2-dataset/bb.npy')

X = (X / 255.).reshape(len(X), 32*32*3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(X_train.shape)
print(y_train.shape)

(811, 3072)
(811,)


In [311]:
net2 = myNetwork()
net2.add_layer(myLayer(32*32*3, 128))
net2.add_layer(relu())
net2.add_layer(myLayer(128, 32))
net2.add_layer(relu())
net2.add_layer(myLayer(32, 3))

for epoch in range(20):
    epoch_loss = []
    for x_batch, y_batch in iterate_minibatches(X_train, y_train, batchsize=1, shuffle=True):

        ### forward ###
        logits = net2.forward(x_batch)

        ### loss calculation ###
        loss = softmax_crossentropy_with_logits(logits, y_batch)
        epoch_loss.append(np.mean(loss))

        ### backward ###
        loss_grad = grad_softmax_crossentropy_with_logits(logits, y_batch)
        net2.backward(loss_grad)

    preds = []
    for x_batch, _ in iterate_minibatches(X_test, y_test, batchsize=1, shuffle=False):
        preds.append(np.argmax(net2.forward(x_batch)))

    print(f'\nEpoch: {epoch + 1} \t Loss: {np.mean(epoch_loss)} \t Test accuracy: {accuracy_score(y_test, preds)} \n')

minibatch iteration: 100%|██████████| 91/91 [00:00<00:00, 3430.85it/s]



Epoch: 1 	 Loss: 1.1003026058451546 	 Test accuracy: 0.43956043956043955 



minibatch iteration: 100%|██████████| 91/91 [00:00<00:00, 3974.03it/s]



Epoch: 2 	 Loss: 1.1008133270539506 	 Test accuracy: 0.43956043956043955 



minibatch iteration: 100%|██████████| 91/91 [00:00<00:00, 3928.42it/s]


...


Epoch: 18 	 Loss: 0.09995121328746581 	 Test accuracy: 0.9340659340659341 



minibatch iteration: 100%|██████████| 91/91 [00:00<00:00, 4605.62it/s]



Epoch: 19 	 Loss: 0.10581186851954756 	 Test accuracy: 0.9120879120879121 



minibatch iteration: 100%|██████████| 91/91 [00:00<00:00, 4117.87it/s]


Epoch: 20 	 Loss: 0.014646831862251504 	 Test accuracy: 0.967032967032967 

